# Mode d’emploi
Un notebook Jupyter est formé
- de cellules qui contiennent du texte (comme celui que vous êtes en train de lire) au format Markdown,
- de cellules grisées qui contiennent du code exécutable (ici en Python).

Pour modifier le code d’une cellule (tout est modifiable) : cliquer ou double-cliquer dans la cellule.
    
Pour exécuter le code d’une cellule (Markdown ou Python) : placer le curseur dans la cellule est **Ctrl+Entrée== (ou la flèche dans le menu).


# 1. Simulation du prélèvement d’un échantillon
On simule le prélèvement d’un échantillon et on renvoie la fréquence de succès.

Les paramètres sont la proportion dans la population et la taille de l’échantillon.


In [ ]:
from random import random                                    # On importe la fonction random : nombre aléatoire entre 0 et 1.

def FrequenceEchantillon(proportion,effectif):
    compte = 0
    for i in range(effectif):
        if random() < proportion:                            # Si le nombre aléatoire est inférieur à la proportion,
            compte += 1                                      # on ajoute 1 au compte.
    return compte/effectif                                   # On renvoie la fréquence de succès.

**Exemple** : On peut voir qu’avec un échantillon de 1000 personnes et une proportion de 51 %, la fréquence observée est souvent en-dessous de 50 % (répéter plusieurs fois **ctrl+entrée** pour voir les _fluctuations d’échantillonnage_).

In [ ]:
FrequenceEchantillon(0.51,1000)

# 2. Calcul d’un intervalle de confiance
Une fois qu’on a observé une fréquence dans un échantillon, on peut déterminer un intervalle de confiance.

On définit plusieurs méthode de calcul d’un intervalle de confiance.

Les paramètres sont la fréquence observée, l’effectif de l’échantillon et le niveau de confiance.

**Wald** : À partir du niveau, on calcule la valeur de $t$, puis on renvoie un couple représentant l’intervalle de confiance.  
Si le niveau n’est pas prévu, on renvoie $(1,0)$ qui est le signe d’une erreur.  
(Je n’ai pas implémenté la formule qui donne la valeur de $t$ dans le cas général parce qu’elle est compliquée)

**Agresti-Coull** : Comme l’intervalle de Wald, mais en ajoutant 2 succès et 2 échecs. L’intervalle est recentré, et son amplitude diminue légèrerement.

In [ ]:
from math import sqrt                 # Import de la fonction racine carrée qui n’est pas native

def Wald(frequence,effectif,niveau):
    if niveau == 90:
        t = 1.645
    elif niveau == 95:
        t = 1.96
    elif niveau == 99:
        t = 2.576
    else:
        return (1,0)
    return (frequence-t*sqrt(frequence*(1-frequence)/effectif),frequence+t*sqrt(frequence*(1-frequence)/effectif))

def AgrestiCoull(frequence,effectif,niveau):
    return Wald((effectif*frequence + 2)/(effectif + 4),effectif+4,niveau)

**Exemple** : On calcule l’intervalle de Wald…

In [ ]:
Wald(0.3,200,95)

**Exemple** … ou l’intervalle de Agresti-Coull

In [ ]:
AgrestiCoull(0.3,200,95)

# 3. Test de recouvrement
On crée une fonction qui
- simule le tirage d’un échantillon,
- calcule l’intervalle de confiance correspondant à la fréquence obtenue,
- dit sous forme de booléen si l’intervalle de confiance recouvre la proportion.

Les paramètres sont
- la proportion à estimer dans la population,
- la taille de l’échantillon,
- le niveau de confiance souhaité,
- la méthode de calcul de l’intervalle de confiance (Wald, Agresti-Coull…) : le langage Python accepte le passage d’une fonction en paramètre.

In [ ]:
def Recouvrement(proportion,effectif,niveau,methode):
    frequence = FrequenceEchantillon(proportion,effectif)    # Simulation d’un tirage d’un échantillon et renvoie de la fréquence
    intervalle = methode(frequence,effectif,niveau)          # Calcul de l’intervalle de confiance avec la méthode souhaitée
    return intervalle[0] <= proportion <= intervalle[1]      # Test du recouvrement et renvoi du booléen correspondant.

**Exemple** : Répéter plusieurs fois Ctrl+Entrée pour voir qu’il n’y a pas toujours recouvrement. Éventuellement, modifier les paramètres.

In [ ]:
Recouvrement(0.3,200,90,Wald)

# 4. Fréquence de recouvrement
On crée une fonction qui réalise un grand nombre de test de recouvrement, et renvoie la fréquence de recouvrement.

Les paramètres sont le nombre de test, la proportion à estimer, la taille de l’échantillon, le niveau de confiance, et la méthode de calcul de l’intervalle de confiance.

In [ ]:
def FrequenceRecouvrement(nbtest,proportion,effectif,niveau,methode):
    compte = 0
    for i in range(nbtest):
        if Recouvrement(proportion,effectif,niveau,methode):
            compte += 1
    return compte/nbtest

**Exemple** : Pour une proportion à estimer de 30 %, des échantillons de taille 100, un niveau de confiance de 90%, et la méthode de Wald, quelle est la fréquence de recouvrement obtenue pour 10000 tests ?

On voit que la fréquence de recouvrement est nettement inférieur à 90%…

In [ ]:
FrequenceRecouvrement(10000,0.3,100,90,Wald)

Vérifier qu’avec l’intervalle d’Agresti-Coull, la fréquence de recouvrement est meilleure.

Changer éventuellement les autres paramètres.

**Exemple** On peut éventuellement changer automatiquement la proportion $p$

In [ ]:
nbpas = 10
for i in range(0,nbpas+1):
    p = i/nbpas
    print(p,FrequenceRecouvrement(100000,p,100,90,Wald))